#### Taipei Parking Scraper 



In [1]:
# === Imports ===
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse
import csv
import sys

# Optional: pandas for convenient CSV export / preview
try:
    import pandas as pd
except Exception:
    pd = None

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "zh-TW,zh;q=0.9,en;q=0.8",
}
TIMEOUT = 20

In [2]:
# === Parameters ===
# 你可以在這裡更改要抓取的頁面與 PageSize
URL = "https://pma.gov.taipei/News.aspx?n=6833ECE829BE5990&sms=504AB58CAE8A1C62&page=17&PageSize=337"

# 輸出的檔名
OUTPUT_CSV = "taipei_parking_page17.csv"

In [3]:
# === Helper Functions ===
def fetch_soup(url: str) -> BeautifulSoup:
    """抓取並回傳 BeautifulSoup 物件。"""
    r = requests.get(url, headers=HEADERS, timeout=TIMEOUT)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")

def norm(txt: str) -> str:
    """去除多餘空白與全形空白。"""
    return (txt or "").replace("\xa0", " ").replace("\u3000", " ").strip()

def extract_rows(soup: BeautifulSoup):
    """
    逐列把 <td data-title="欄名">值</td> 組成 dict。
    這頁的 <td> 都有 data-title，所以用它來定位欄位最保險。
    只保留有「停車場名稱」的列。
    """
    rows = []
    # 掃所有 tbody（有時頁面不只一個表）
    for tbody in soup.find_all("tbody"):
        for tr in tbody.find_all("tr"):
            tds = tr.find_all("td")
            if not tds:
                continue
            row = {}
            for td in tds:
                key = norm(td.get("data-title") or "")
                if not key:
                    continue
                val = norm(td.get_text(" ", strip=True))
                row[key] = val
            if row.get("停車場名稱"):
                rows.append(row)
    return rows

In [4]:
# === Field Mapping ===
KEYMAP = {
    "name":      ["停車場名稱"],
    "rate":      ["費率", "收費標準"],
    "address":   ["停車場地址", "地址"],
    "phone":     ["停車場電話", "電話", "聯絡電話"],
    "open_time": ["開放時間", "營業時間", "開放時段"],
    "car_spaces": ["小車", "小型車", "小客車"],
    "moto_spaces": ["機車", "機車位"],
    "truck_spaces": ["大車", "大客車", "大貨車"],
    "district": ["行政區"],
    "type": ["停車場位置點"],
}

def pick(row: dict, keys: list):
    for k in keys:
        if k in row and row[k] != "":
            return row[k]
    return ""

def map_fields(row: dict) -> dict:
    return {
        "停車場名稱": pick(row, KEYMAP["name"]),
        "費率": pick(row, KEYMAP["rate"]),
        "停車場地址": pick(row, KEYMAP["address"]),
        "電話": pick(row, KEYMAP["phone"]),
        "開放時間": pick(row, KEYMAP["open_time"]),
        "汽車車位數": pick(row, KEYMAP["car_spaces"]),
        "機車車位數": pick(row, KEYMAP["moto_spaces"]),
        "大車車位數": pick(row, KEYMAP["truck_spaces"]),
        "行政區": pick(row, KEYMAP["district"]),
        "位置型態": pick(row, KEYMAP["type"]),
    }

In [5]:
# === Save CSV ===
def save_csv(records, path="taipei_parking_page17.csv"):
    """優先用 pandas；若無則退回內建 csv。"""
    if not records:
        print("沒有可儲存的資料。")
        return

    if pd is not None:
        df = pd.DataFrame(records)
        df.to_csv(path, index=False, encoding="utf-8")
        print(f"已輸出 CSV：{path}（使用 pandas）")
    else:
        cols = list(records[0].keys())
        with open(path, "w", newline="", encoding="utf-8") as f:
            w = csv.DictWriter(f, fieldnames=cols)
            w.writeheader()
            for r in records:
                w.writerow(r)
        print(f"已輸出 CSV：{path}（使用內建 csv）")

In [10]:
# === Run ===
# 抓取與解析
soup = fetch_soup(URL)
raw_rows = extract_rows(soup)

# 欄位正規化
data = [map_fields(r) for r in raw_rows]

# 檢視前幾筆
for i, row in enumerate(data[:5], 1):
    preview = [
        row.get('停車場名稱', ''),
        row.get('行政區', ''),
        row.get('停車場地址', ''),
        f"小車:{row.get('汽車車位數', '')} 機車:{row.get('機車車位數', '')}",
        row.get('費率', ''),
        row.get('開放時間', ''),
        row.get('電話', ''),
    ]
    print(f"[{i}] " + "｜".join(map(str, preview)))

print("總筆數：", len(data))



[1] 遼寧街185巷機車平面停車場｜臺北市中山區｜遼寧街185巷17號對面｜小車: 機車:35｜計次20元(凡跨越每日零時起重行起算)｜9-17｜
[2] 大業路527巷平面停車場｜臺北市北投區｜大業路527巷近捷運復興崗站｜小車:14 機車:10｜小型車計次50元，機車免費。｜9-17｜
[3] 平陽街18號機車平面停車場｜臺北市大同區｜平陽街18號旁｜小車: 機車:23｜計時10元，當日當次最高上限30元(隔日另計)。｜9-20｜
[4] 中華路二段409巷平面停車場｜臺北市中正區｜中華路2段409巷1號對面｜小車:5 機車:｜計時30元｜9-17｜
[5] 南港公園平面停車場｜臺北市南港區｜東新街170號(南港公園大門旁)｜小車:84 機車:｜計時20元｜24小時｜29441489
總筆數： 337


In [7]:
# === Save to CSV ===
save_csv(data, OUTPUT_CSV)

已輸出 CSV：taipei_parking_page17.csv（使用 pandas）


In [8]:
import requests

PLACES_URL = "https://maps.googleapis.com/maps/api/place/textsearch/json"

def get_lat_lng_place(query, api_key):
    params = {
        "query": query,
        "key": api_key,
        "language": "zh-TW",  # 回傳繁體中文
        "region": "tw"        # 偏向台灣地區
    }
    r = requests.get(PLACES_URL, params=params, timeout=10)
    data = r.json()
    
    if data.get("status") == "OK" and data.get("results"):
        loc = data["results"][0]["geometry"]["location"]
        return loc["lat"], loc["lng"]
    else:
        raise RuntimeError(f"查詢失敗：{data.get('status')} - {data.get('error_message')}")

# === 測試 ===
with open("api.txt", "r", encoding="utf-8") as f:
    api_key = f.read().strip()

query = "重新家樂福 12mini"
try:
    lat, lng = get_lat_lng_place(query, api_key)
    print(f"經緯度：{lat}, {lng}")
except Exception as e:
    print("查詢失敗：", e)


經緯度：25.0431845, 121.4678297


In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv('taipei_parking_page17.csv')


# 先新增空欄位，先放 NaN
df["latitude"] = np.nan
df["longitude"] = np.nan

for idx, row in df.iterrows():
    name = row["停車場名稱"]
    try:
        lat, lng = get_lat_lng_place(name, api_key)
        # 存回 DataFrame
        df.at[idx, "latitude"] = lat
        df.at[idx, "longitude"] = lng
        print(f"{name} → 經緯度：{lat}, {lng}")
    except Exception as e:
        print(f"{name} 查詢失敗：{e}")


        

遼寧街185巷機車平面停車場 → 經緯度：25.0537072, 121.5427812
大業路527巷平面停車場 → 經緯度：25.1356749, 121.4946365
平陽街18號機車平面停車場 → 經緯度：22.6406602, 120.3072516
中華路二段409巷平面停車場 → 經緯度：22.7259484, 120.3525723
南港公園平面停車場 → 經緯度：25.0457639, 121.5913917
洲美公園臨時平面停車場 → 經緯度：25.1002147, 121.505771
北投製片廠平面停車場 → 經緯度：25.1397346, 121.487086
內湖國中地下停車場 → 經緯度：25.0770945, 121.5891588
文林橋下平面停車場 → 經緯度：25.101787, 121.5166173
民權東路3段153巷西側平面停車場 → 經緯度：25.062314, 121.5464542
民權東路3段197巷口東側平面停車場 → 經緯度：25.062314, 121.5464542
三合街1段82巷62弄平面停車場 → 經緯度：25.1279102, 121.5053116
三合街1段82巷86弄平面停車場 → 經緯度：25.1279102, 121.5053116
興隆A區社會住宅地下停車場 → 經緯度：24.9881558, 121.5625456
興德路62巷平面停車場 → 經緯度：25.000642, 121.551241
大安國中地下停車場 → 經緯度：25.0299188, 121.5461616
環南堤外平面停車場 → 經緯度：25.0288489, 121.4880539
杭州南路2段51巷平面停車場 → 經緯度：25.0329663, 121.5232941
廣慈3區社會住宅地下停車場 → 經緯度：25.0392794, 121.5828103
吳興街220巷59弄平面停車場 → 經緯度：25.0268659, 121.5604614
臨沂街71巷平面停車場 → 經緯度：25.036734, 121.5288569
福國路(洲美段二)橋下平面停車場 → 經緯度：25.1207239, 121.4990794
民權東路三段197巷口西側平面停車場 → 經緯度：25.062314, 121.5464542

In [11]:
df.to_csv("taipei_parking_with_lan.csv", index=False, encoding="utf-8-sig")
